The traffic accident data is the key to this analysis. To make sure it's current, instead of downloading a static .csv file it, along with several other datasets from the City of Nashville's OpenData portal, will be periodically downloaded and updated via APIs.

To avoid too many pull requests, all of the data will be pulled here and the relevant information will be exported for use in a separate notebook for EDA.

** This notebook will be revisited and edited as data needs may change for this analysis.

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
# Needed to pip install sodapy first
from sodapy import Socrata
from geopy.geocoders import Nominatim

**Data Source:** https://data.nashville.gov/Police/Traffic-Accidents/6v6w-hpcw/about_data

In [2]:
client = Socrata("data.nashville.gov", None)

In [3]:
crashes_export = client.get("6v6w-hpcw", limit=200000)
crashes_raw = pd.DataFrame.from_records(crashes_export)
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,property_damage,hit_and_run,reporting_officer,collision_type,collision_type_description,...,precinct,lat,long,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,harmfulcodes,harmfuldescriptions
0,20240137582,2024-02-27T18:05:00.000,1,0,0,True,False,311327,0,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,...,EAST,36.1756,-86.7615,"{'type': 'Point', 'coordinates': [-86.7615, 36...",1,1,13,4,NaN,NaN
1,20240137450,2024-02-27T17:15:00.000,2,1,0,NaN,False,311327,98,OTHER,...,EAST,36.2062,-86.7729,"{'type': 'Point', 'coordinates': [-86.7729, 36...",1,1,13,16,12;14,MOTOR VEHICLE IN TRANSPORT;PARKED MOTOR VEHICLE
2,20240137422,2024-02-27T17:10:00.000,2,0,0,NaN,False,320590,6,SIDESWIPE - OPPOSITE DIRECTION,...,NORTH,36.1847,-86.8157,"{'type': 'Point', 'coordinates': [-86.8157, 36...",1,1,25,51,12,MOTOR VEHICLE IN TRANSPORT
3,20240137264,2024-02-27T14:53:00.000,2,0,0,NaN,False,4008024,5,SIDESWIPE - SAME DIRECTION,...,CENTRA,36.1519,-86.7866,"{'type': 'Point', 'coordinates': [-86.7866, 36...",1,1,20,40,12,MOTOR VEHICLE IN TRANSPORT
4,20240137215,2024-02-27T15:36:00.000,2,1,0,NaN,False,4000973,4,ANGLE,...,MIDTOW,36.1219,-86.7602,"{'type': 'Point', 'coordinates': [-86.7602, 36...",1,2,18,3,12,MOTOR VEHICLE IN TRANSPORT


In [4]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180680 entries, 0 to 180679
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   accident_number              180680 non-null  object
 1   date_and_time                180680 non-null  object
 2   number_of_motor_vehicles     180679 non-null  object
 3   number_of_injuries           180680 non-null  object
 4   number_of_fatalities         180680 non-null  object
 5   property_damage              13727 non-null   object
 6   hit_and_run                  180666 non-null  object
 7   reporting_officer            180669 non-null  object
 8   collision_type               180670 non-null  object
 9   collision_type_description   180670 non-null  object
 10  weather                      174621 non-null  object
 11  weather_description          174621 non-null  object
 12  illuaccidemination           180398 non-null  object
 13  illumination_d

Now that the DataFrame has been created, some columns can be removed as they're not relevant to this analysis.

In [5]:
crashes_raw = crashes_raw.drop(crashes_raw.columns[[7, 8, 10, 12, 23, 24, 25, 26, 27]], axis=1)

In [6]:
crashes_raw['date'] = pd.to_datetime(crashes_raw['date_and_time']).dt.date
crashes_raw = crashes_raw.astype({'number_of_motor_vehicles': 'float',
                    'number_of_injuries': 'float',
                    'number_of_fatalities': 'float',
                    'hit_and_run': 'bool',
                    'property_damage': 'bool'})

In [7]:
crashes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180680 entries, 0 to 180679
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   accident_number             180680 non-null  object 
 1   date_and_time               180680 non-null  object 
 2   number_of_motor_vehicles    180679 non-null  float64
 3   number_of_injuries          180680 non-null  float64
 4   number_of_fatalities        180680 non-null  float64
 5   property_damage             180680 non-null  bool   
 6   hit_and_run                 180680 non-null  bool   
 7   collision_type_description  180670 non-null  object 
 8   weather_description         174621 non-null  object 
 9   illumination_description    180398 non-null  object 
 10  street_address              180674 non-null  object 
 11  city                        180680 non-null  object 
 12  state                       180679 non-null  object 
 13  zip           

For some initial cleaning, all text fields should be converted to the same case, in this case UPPER

In [8]:
crashes_raw['collision_type_description'] = crashes_raw['collision_type_description'].astype(str).str.upper()
crashes_raw['illumination_description'] = crashes_raw['illumination_description'].astype(str).str.upper()
crashes_raw['harmfuldescriptions'] = crashes_raw['harmfuldescriptions'].astype(str).str.upper()
crashes_raw['street_address'] = crashes_raw['street_address'].astype(str).str.upper()
crashes_raw['city'] = crashes_raw['city'].astype(str).str.upper()
crashes_raw['state'] = crashes_raw['state'].astype(str).str.upper()
crashes_raw['precinct'] = crashes_raw['precinct'].astype(str).str.upper()
crashes_raw['weather_description'] = crashes_raw['weather_description'].astype(str).str.upper()
crashes_raw['property_damage'] = crashes_raw['property_damage'].astype(str).str.upper()

In [9]:
crashes_raw.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,property_damage,hit_and_run,collision_type_description,weather_description,illumination_description,...,city,state,zip,rpa,precinct,lat,long,mapped_location,harmfuldescriptions,date
0,20240137582,2024-02-27T18:05:00.000,1.0,0.0,0.0,TRUE,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DAYLIGHT,...,NASHVILLE,TN,37206,1999,EAST,36.1756,-86.7615,"{'type': 'Point', 'coordinates': [-86.7615, 36...",NAN,2024-02-27
1,20240137450,2024-02-27T17:15:00.000,2.0,1.0,0.0,TRUE,False,OTHER,CLEAR,DAYLIGHT,...,NASHVILLE,TN,37207,1863,EAST,36.2062,-86.7729,"{'type': 'Point', 'coordinates': [-86.7729, 36...",MOTOR VEHICLE IN TRANSPORT;PARKED MOTOR VEHICLE,2024-02-27
2,20240137422,2024-02-27T17:10:00.000,2.0,0.0,0.0,TRUE,False,SIDESWIPE - OPPOSITE DIRECTION,CLEAR,DAYLIGHT,...,NASHVILLE,TN,37208,4417,NORTH,36.1847,-86.8157,"{'type': 'Point', 'coordinates': [-86.8157, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-27
3,20240137264,2024-02-27T14:53:00.000,2.0,0.0,0.0,TRUE,False,SIDESWIPE - SAME DIRECTION,CLEAR,DAYLIGHT,...,NASHVILLE,TN,37203,40041,CENTRA,36.1519,-86.7866,"{'type': 'Point', 'coordinates': [-86.7866, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-27
4,20240137215,2024-02-27T15:36:00.000,2.0,1.0,0.0,TRUE,False,ANGLE,CLOUDY,DAYLIGHT,...,NASHVILLE,TN,37211,81060,MIDTOW,36.1219,-86.7602,"{'type': 'Point', 'coordinates': [-86.7602, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-27


Last, this analysis is only concerned with crashes in the vicinity of Main Street / Gallatin Avenue / Gallatin Pike in East Nashville & Madison.

In [10]:
crashes = crashes_raw[crashes_raw['street_address'].str.contains('GALLATIN|MAIN')]
crashes.head()

,accident_number,date_and_time,number_of_motor_vehicles,number_of_injuries,number_of_fatalities,property_damage,hit_and_run,collision_type_description,weather_description,illumination_description,...,city,state,zip,rpa,precinct,lat,long,mapped_location,harmfuldescriptions,date
5,20240136898,2024-02-27T12:20:00.000,2.0,0.0,0.0,TRUE,True,SIDESWIPE - SAME DIRECTION,CLOUDY,DAYLIGHT,...,NASHVILLE,TN,37206,1401,EAST,36.1971,-86.742,"{'type': 'Point', 'coordinates': [-86.742, 36....",MOTOR VEHICLE IN TRANSPORT,2024-02-27
19,20240135639,2024-02-27T01:29:00.000,2.0,0.0,0.0,TRUE,True,FRONT TO REAR,CLEAR,DARK - LIGHTED,...,NASHVILLE,TN,37206,1131,EAST,36.1763,-86.7564,"{'type': 'Point', 'coordinates': [-86.7564, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-27
22,20240134815,2024-02-26T19:40:00.000,2.0,0.0,0.0,TRUE,False,SIDESWIPE - SAME DIRECTION,CLEAR,DARK - LIGHTED,...,MADISON,TN,37115,1783,MADISO,36.2562,-86.7144,"{'type': 'Point', 'coordinates': [-86.7144, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-26
30,20240134529,2024-02-26T16:49:00.000,2.0,0.0,0.0,TRUE,False,ANGLE,CLEAR,DAYLIGHT,...,NASHVILLE,TN,37206,1301,EAST,36.1843,-86.7482,"{'type': 'Point', 'coordinates': [-86.7482, 36...",MOTOR VEHICLE IN TRANSPORT,2024-02-26
45,20240133801,2024-02-26T12:00:00.000,1.0,1.0,0.0,TRUE,False,NOT COLLISION W/MOTOR VEHICLE-TRANSPORT,CLEAR,DAYLIGHT,...,MADISON,TN,37115,1605,MADISO,36.2604,-86.7129,"{'type': 'Point', 'coordinates': [-86.7129, 36...",NAN,2024-02-26


The resulting table is exported to a .csv file for use in the EDA notebook.

In [11]:
crashes.to_csv('../data/clean/crashes.csv')

Now repeat the process for active ROW permits.<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Active-Right-of-Way-Permits/a5tp-4w2v/about_data

In [12]:
row_permits_export = client.get("a5tp-4w2v", limit=2000000)
row_permits_raw = pd.DataFrame.from_records(row_permits_export)
row_permits_raw.head()

,initiated_date,permit,permit_type,permit_description,status,on_street,from_street,to_street,location_address,city,...,company,days_to_work,latitude,longitude,mapped_location,:@computed_region_wvby_4s8j,:@computed_region_3aw5_2wv7,:@computed_region_p6sk_2acq,:@computed_region_gxvr_9jxz,:@computed_region_b9k3_hpc2
0,2023-03-08T08:19:14.000,202308727,DRIVEWAY PERMIT,TEMPORARY ENTRANCE TO CONSTRUCTION SITE,ACTIVE,OLD HARDING PIKE,CLOFTON DR,HARPETH PKWY W,OLD HARDING PIKE,NASHVILLE,...,PIEDMONT NATURAL GAS,Monday Tuesday Wednesday Thursday Friday Satur...,36.0639239825514,-86.9404312907038,"{'type': 'Point', 'coordinates': [-86.94043129...",1,2,31,32,5
1,2023-05-15T10:27:04.000,202317822,EXCAVATION PERMIT,PLACING ANCHOR\nJOB# A02G791 (NN)\nSTEP: 1.7,ACTIVE,META DR,WESTCREST DR,ELLENWOOD DR,META DR,NASHVILLE,...,STAR CONSTRUCTION LLC,Tuesday Wednesday Thursday Friday Saturday,36.0594833908917,-86.7349955972504,"{'type': 'Point', 'coordinates': [-86.73499559...",1,1,21,3,3
2,2023-06-06T14:06:00.000,202320687,DRIVEWAY PERMIT,NEW DRIVEWAY,ACTIVE,849 BATTLE RD,MID-BLOCK,BATTLE RIDGE LN,849 BATTLE RD,NOLENSVILLE,...,NaN,Monday Tuesday Wednesday Thursday Friday Satur...,35.9746479630669,-86.6363976246735,"{'type': 'Point', 'coordinates': [-86.63639762...",1,2,29,18,3
3,2023-06-26T08:38:09.000,202323047,DRIVEWAY PERMIT,CONSTRUCTION OF A TEMPORARY DRIVEWAY TO ENTER ...,ACTIVE,MCCRORY LN,NEWSOM STATION RD,I40 RAMP,MCCRORY LN,NASHVILLE,...,Piedmont Natural Gas,Monday Tuesday Wednesday Thursday Friday Satur...,36.0787530201599,-87.0224639441398,"{'type': 'Point', 'coordinates': [-87.02246394...",1,2,35,32,5
4,2023-07-10T14:37:22.000,202324681,DRIVEWAY PERMIT,MODIFACATION TO APRONS ENTRANCE,ACTIVE,417 417 WINDSOR DR,BLACKBURN AVE,HARDING PL,417 417 WINDSOR DR,NASHVILLE,...,THE OARENT COMPANY INC,Monday Tuesday Wednesday Thursday Friday Satur...,36.1014334222627,-86.8669816953158,"{'type': 'Point', 'coordinates': [-86.86698169...",1,1,32,5,6


In [13]:
row_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   initiated_date               16 non-null     object
 1   permit                       16 non-null     object
 2   permit_type                  16 non-null     object
 3   permit_description           16 non-null     object
 4   status                       16 non-null     object
 5   on_street                    16 non-null     object
 6   from_street                  16 non-null     object
 7   to_street                    16 non-null     object
 8   location_address             16 non-null     object
 9   city                         16 non-null     object
 10  zip                          16 non-null     object
 11  scope                        16 non-null     object
 12  scheduled_start              16 non-null     object
 13  scheduled_end                16 non-n

In [14]:
row_permits_raw = row_permits_raw.drop(row_permits_raw.columns[[4, 22, 23, 24, 25, 26]], axis=1)

In [15]:
row_permits_raw['initiated_date'] = pd.to_datetime(row_permits_raw['initiated_date'])
row_permits_raw['scheduled_start'] = pd.to_datetime(row_permits_raw['scheduled_start'])
row_permits_raw['scheduled_end'] = pd.to_datetime(row_permits_raw['scheduled_end'])

In [16]:
row_permits_raw['permit_type'] = row_permits_raw['permit_type'].astype(str).str.upper()
row_permits_raw['permit_description'] = row_permits_raw['permit_description'].astype(str).str.upper()
row_permits_raw['on_street'] = row_permits_raw['on_street'].astype(str).str.upper()
row_permits_raw['from_street'] = row_permits_raw['from_street'].astype(str).str.upper()
row_permits_raw['to_street'] = row_permits_raw['to_street'].astype(str).str.upper()
row_permits_raw['location_address'] = row_permits_raw['location_address'].astype(str).str.upper()
row_permits_raw['city'] = row_permits_raw['city'].astype(str).str.upper()
row_permits_raw['scope'] = row_permits_raw['scope'].astype(str).str.upper()
row_permits_raw['permit_applicant_name'] = row_permits_raw['permit_applicant_name'].astype(str).str.upper()
row_permits_raw['council_district'] = row_permits_raw['council_district'].astype(str).str.upper()
row_permits_raw['conditions_traffic_control'] = row_permits_raw['conditions_traffic_control'].astype(str).str.upper()
row_permits_raw['company'] = row_permits_raw['company'].astype(str).str.upper()

In [17]:
row_permits = row_permits_raw[row_permits_raw['on_street'].str.contains('GALLATIN|MAIN')]
row_permits

,initiated_date,permit,permit_type,permit_description,on_street,from_street,to_street,location_address,city,zip,...,scheduled_start,scheduled_end,permit_applicant_name,council_district,conditions_traffic_control,company,days_to_work,latitude,longitude,mapped_location


In [18]:
row_permits.to_csv('../data/clean/row_permits.csv')

...and Nashville 311 calls<br><br>
Source: https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh/about_data

In [19]:
nash_311_export = client.get("7qhx-rexh", limit=2000000)
nash_311_raw = pd.DataFrame.from_records(nash_311_export)
nash_311_raw.head()

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,...,:@computed_region_v67z_xm3t,:@computed_region_kh5x_g7w5,:@computed_region_cfa7_hbpz,:@computed_region_sjpq_96s8,:@computed_region_gisn_y5cm,:@computed_region_b9k3_hpc2,oem_id,contact_type,parent_case,preferred_language
0,966190,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-05-04T17:54:30.000,2022-05-04T17:54:30.000,Phone,False,True,...,2,3,71,1527,4,2,NaN,NaN,NaN,NaN
1,932934,Closed,Public Safety,Abandoned Vehicles,Abandoned Vehicles,2022-04-01T21:01:39.000,2022-04-05T11:29:54.000,Phone,False,False,...,1,9,1018,15550,4,1,NaN,NaN,NaN,NaN
2,932939,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Report a Damaged Cart,2022-04-01T21:04:50.000,2022-04-06T20:14:15.000,Phone,False,False,...,8,33,258,24062,7,8,NaN,NaN,NaN,NaN
3,932935,Closed,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,Resolved by hubNashville on First Call,2022-04-01T21:01:47.000,2022-04-01T21:01:47.000,Phone,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,932938,Closed,Public Safety,Request Non-Emergency Police Assistance,NaN,2022-04-01T21:03:10.000,2022-04-01T22:04:31.000,Phone,False,False,...,7,13,50,30406,1,7,NaN,NaN,NaN,NaN


In [20]:
nash_311_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442835 entries, 0 to 1442834
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   case_number                  1442835 non-null  object
 1   status                       1442835 non-null  object
 2   case_request                 1442820 non-null  object
 3   case_subrequest              1441495 non-null  object
 4   additional_subrequest        1353202 non-null  object
 5   date_time_opened             1442835 non-null  object
 6   date_time_closed             1428159 non-null  object
 7   case_origin                  1442835 non-null  object
 8   state_issue                  1442835 non-null  object
 9   closed_when_created          1442835 non-null  object
 10  incident_address             981975 non-null   object
 11  incident_city                965295 non-null   object
 12  incident_council_district    966934 non-null   object
 1

In [21]:
nash_311_raw = nash_311_raw.drop(nash_311_raw.columns[[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]], axis=1)

In [22]:
nash_311_raw['date_time_opened'] = pd.to_datetime(nash_311_raw['date_time_opened'])
nash_311_raw['date_time_closed'] = pd.to_datetime(nash_311_raw['date_time_closed'])

In [23]:
nash_311_raw['status'] = nash_311_raw['status'].astype(str).str.upper()
nash_311_raw['case_request'] = nash_311_raw['case_request'].astype(str).str.upper()
nash_311_raw['case_subrequest'] = nash_311_raw['case_subrequest'].astype(str).str.upper()
nash_311_raw['additional_subrequest'] = nash_311_raw['additional_subrequest'].astype(str).str.upper()
nash_311_raw['case_origin'] = nash_311_raw['case_origin'].astype(str).str.upper()
nash_311_raw['state_issue'] = nash_311_raw['state_issue'].astype(str).str.upper()
nash_311_raw['incident_address'] = nash_311_raw['incident_address'].astype(str).str.upper()
nash_311_raw['incident_city'] = nash_311_raw['incident_city'].astype(str).str.upper()
nash_311_raw['incident_council_district'] = nash_311_raw['incident_council_district'].astype(str).str.upper()
nash_311_raw['contact_type'] = nash_311_raw['contact_type'].astype(str).str.upper()
nash_311_raw['parent_case'] = nash_311_raw['parent_case'].astype(str).str.upper()
nash_311_raw['preferred_language'] = nash_311_raw['preferred_language'].astype(str).str.upper()

In [24]:
nash_311 = nash_311_raw[nash_311_raw['incident_address'].str.contains('GALLATIN|MAIN')]
nash_311

,case_number,status,case_request,case_subrequest,additional_subrequest,date_time_opened,date_time_closed,case_origin,state_issue,closed_when_created,incident_address,incident_city,incident_council_district,incident_zip_code,latitude,longitude,mapped_location,contact_type,parent_case,preferred_language
147,706362,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2021-06-14 03:41:00,2021-06-14 03:41:00,MOBILE APP,FALSE,TRUE,609 GALLATIN PIKE S,MADISON,9,37115,36.2568189,-86.7137494,"{'latitude': '36.2568189', 'longitude': '-86.7...",NAN,NAN,NAN
196,705132,CLOSED,"TRASH, RECYCLING & LITTER",MISSED PICKUP - RECYCLE CART SERVICE,RECYCLE - BACKDOOR,2021-06-11 02:40:00,2021-06-23 15:29:00,HUBNASHVILLE COMMUNITY,FALSE,FALSE,630 MAINSTREAM DR,NASHVILLE,2,37228,36.19805001,-86.81561503,"{'latitude': '36.19805001', 'longitude': '-86....",NAN,NAN,NAN
238,932992,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:07:35,2022-04-01 22:07:35,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
264,707700,CLOSED,"STREETS, ROADS & SIDEWALKS",TRAFFIC LIGHT TIMING,TRAFFIC LIGHT TIMING,2021-06-15 14:40:00,2021-06-28 20:16:00,PHONE,FALSE,FALSE,CONFERENCE DR & GALLATIN PIKE N,MADISON,10,37115,36.3027491,-86.6918546,"{'latitude': '36.3027491', 'longitude': '-86.6...",NAN,NAN,NAN
424,932994,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2022-04-01 22:11:51,2022-04-01 22:11:51,MOBILE APP,FALSE,True,2232 GALLATIN PIKE N,MADISON,10,37115,36.3078416,-86.6849309,"{'latitude': '36.3078416', 'longitude': '-86.6...",NAN,NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442101,1479420,CLOSED,"TRASH, RECYCLING & LITTER",UNWANTED RECYCLING OR TRASH,UNWANTED RECYCLING,2024-02-08 15:24:21,2024-02-09 20:15:09,PHONE,FALSE,False,301 GALLATIN AVE,NASHVILLE,6,37206,36.1800033,-86.749622,"{'latitude': '36.1800033', 'longitude': '-86.7...",NAN,NAN,NAN
1442536,1479888,CLOSED,PUBLIC SAFETY,CONTROL NUMBER REQUEST FOR TOWING,CONTROL NUMBER REQUEST FOR TOWING,2024-02-08 20:26:41,2024-02-08 20:26:41,MOBILE APP,FALSE,True,2200 GALLATIN PIKE N,MADISON,10,37115,36.3072083608358,-86.68642144650221,"{'latitude': '36.3072083608358', 'longitude': ...",NAN,NAN,NAN
1442755,1480151,PENDING WO COMPLETION,"STREETS, ROADS & SIDEWALKS",TRAFFIC LIGHT ISSUE,TRAFFIC LIGHT ISSUE,2024-02-08 23:33:12,NaT,MOBILE APP,FALSE,False,3 MAIN ST,NASHVILLE,19,37213,36.16854116347093,-86.77638038992882,"{'latitude': '36.16854116347093', 'longitude':...",RESIDENT,NAN,NAN
1442761,1480162,CLOSED,"STREETS, ROADS & SIDEWALKS",POTHOLES,ROADWAY,2024-02-08 23:44:44,2024-02-09 22:12:31,MOBILE APP,FALSE,False,4713 GALLATIN PIKE,NASHVILLE,7,37216,36.23080684307148,-86.72498524188995,"{'latitude': '36.23080684307148', 'longitude':...",NAN,NAN,NAN


In [25]:
nash_311.to_csv('../data/clean/nash_311.csv')

...and building permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7/about_data

In [26]:
bdlg_permits_export = client.get("3h5w-q8b7", limit=2000000)
bdlg_permits_raw = pd.DataFrame.from_records(bdlg_permits_export)
bdlg_permits_raw.head()

,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,state,...,:@computed_region_gisn_y5cm,:@computed_region_v3ji_vzam,:@computed_region_c9xn_skx3,:@computed_region_sjpq_96s8,:@computed_region_kh5x_g7w5,:@computed_region_yf9r_ed6g,:@computed_region_fvtq_wnma,:@computed_region_p6sk_2acq,:@computed_region_b9k3_hpc2,:@computed_region_gxvr_9jxz
0,2019040337,Building Commercial - New,"Kennel / Stable, Kennel / Pound",08000007300,2019-07-09T00:00:00.000,2021-02-24T00:00:00.000,110889,1016 CAMILLA CALDWELL LN,NASHVILLE,TN,...,6,2005,1,16168,1,1,23,1,1,34
1,2020005572,Building Commercial - Tenant Finish Out,"Medical Office, Professional Services",16100016300,2020-01-27T00:00:00.000,2021-03-11T00:00:00.000,166050,5705 A NOLENSVILLE PIKE,NASHVILLE,TN,...,5,2005,25,15803,25,25,46,21,3,3
2,2020010440,Building Commercial - Rehab,"Retail, Department / Retail Stores",10700005000,2020-02-18T00:00:00.000,2021-03-16T00:00:00.000,242277,1 TERMINAL DR,NASHVILLE,TN,...,3,2005,18,15807,18,18,44,6,9,38
3,2020023897,Building Commercial - Structural Frame,Automobile Parking - Structure,093050A00200CO,2020-04-15T00:00:00.000,2021-04-09T00:00:00.000,61052744,201 PLATFORM WAY N,NASHVILLE,TN,...,1,2005,13,30406,13,13,25,20,7,40
4,2020025536,Building Commercial - New,"College / University, Educational > 12Th",08000002000,2020-04-22T00:00:00.000,2021-08-10T00:00:00.000,1100000,1200 COUNTY HOSPITAL RD,NASHVILLE,TN,...,6,2005,1,16168,1,1,23,1,1,34


In [27]:
bdlg_permits_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33726 entries, 0 to 33725
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       33726 non-null  object
 1   permit_type_description      33726 non-null  object
 2   permit_subtype_description   33726 non-null  object
 3   parcel                       33726 non-null  object
 4   date_entered                 33726 non-null  object
 5   date_issued                  33726 non-null  object
 6   const_cost                   33725 non-null  object
 7   address                      33726 non-null  object
 8   city                         33725 non-null  object
 9   state                        33726 non-null  object
 10  zip                          33725 non-null  object
 11  subdivision_lot              33725 non-null  object
 12  contact                      33726 non-null  object
 13  per_ty                       33

In [28]:
bdlg_permits_raw = bdlg_permits_raw.drop(bdlg_permits_raw.columns[[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]], axis=1)

In [29]:
bdlg_permits_raw['date_entered'] = pd.to_datetime(bdlg_permits_raw['date_entered'])
bdlg_permits_raw['date_issued'] = pd.to_datetime(bdlg_permits_raw['date_issued'])

In [30]:
bdlg_permits_raw['permit_type_description'] = bdlg_permits_raw['permit_type_description'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype_description'] = bdlg_permits_raw['permit_subtype_description'].astype(str).str.upper()
bdlg_permits_raw['address'] = bdlg_permits_raw['address'].astype(str).str.upper()
bdlg_permits_raw['city'] = bdlg_permits_raw['city'].astype(str).str.upper()
bdlg_permits_raw['state'] = bdlg_permits_raw['state'].astype(str).str.upper()
bdlg_permits_raw['contact'] = bdlg_permits_raw['contact'].astype(str).str.upper()
bdlg_permits_raw['per_ty'] = bdlg_permits_raw['per_ty'].astype(str).str.upper()
bdlg_permits_raw['permit_subtype'] = bdlg_permits_raw['permit_subtype'].astype(str).str.upper()
bdlg_permits_raw['purpose'] = bdlg_permits_raw['purpose'].astype(str).str.upper()
bdlg_permits_raw['council_dist'] = bdlg_permits_raw['council_dist'].astype(str).str.upper()
bdlg_permits_raw['census_tract'] = bdlg_permits_raw['census_tract'].astype(str).str.upper()
bdlg_permits_raw['case_status'] = bdlg_permits_raw['case_status'].astype(str).str.upper()

In [31]:
bdlg_permits = bdlg_permits_raw[bdlg_permits_raw['address'].str.contains('GALLATIN|MAIN')].reset_index()
bdlg_permits

,index,permit,permit_type_description,permit_subtype_description,parcel,date_entered,date_issued,const_cost,address,city,...,subdivision_lot,contact,per_ty,permit_subtype,ivr_trk,purpose,council_dist,census_tract,case_status,mapped_location
0,47,2021006138,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,07213038500,2021-01-29,2021-02-26,3560,2500 GALLATIN AVE,NASHVILLE,...,LOT 1 CHESTER AVE SUB,TRI STAR TRANSPORT LLC,CASN,CAZ02A001,3920404,TO ERECT 90SF FACE REPLACEMENT ON EXISTING PYL...,6,37011700,EXPIRED,"{'latitude': '36.195368006', 'longitude': '-86..."
1,73,2021011848,BUILDING COMMERCIAL - NEW,"MULTIFAMILY, APT / TWNHOME > 5 UNIT BLDG",08212031300,2021-02-27,2021-09-09,2025671,918 MAIN ST,NASHVILLE,...,PT LOTS 5 & 6 R W BROWN ADDN,"POWELL ARCHITECTURE & BUILDING STUDIO, LLC",CACN,CAA03R399,3933909,"MUG-A, UDO, MDHA, UZO TO CONSTRUCT MIXED USE ...",6,37019200,DONE,"{'latitude': '36.176308019', 'longitude': '-86..."
2,164,2021024771,BUILDING COMMERCIAL REHAB STORM DAMAGE,"RETAIL, DEPARTMENT / RETAIL STORES",08216001700,2021-04-21,2021-08-03,50000,715 MAIN ST,NASHVILLE,...,P/O LOTS 8 & 9 W. L. FOSTER 1ST ADDN.,"CRAFTLOGIC CONSTRUCTION, INC.",CACL,CAF20M001,3970890,***MARCH 3 2020 TORNADO DAMAGE*** TO REPLAC...,5,37011900,EXPIRED,"{'latitude': '36.175373018', 'longitude': '-86..."
3,368,2021058673,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,02600012300,2021-09-13,2021-09-27,3585,2180 GALLATIN PIKE N,MADISON,...,LOT 4 RIVERGATE PROPERTIES LTD. REV,JOSLIN SIGN & MAINTENANCE CO INC,CASN,CAZ02A001,4037453,TO ERECT 25SF ILLUMINATED WALL SIGN FOR AMES D...,10,37010401,EXPIRED,"{'latitude': '36.305128986', 'longitude': '-86..."
4,393,2021061547,BUILDING SIGN PERMIT,BILLBOARD,04216019300,2021-09-24,2021-10-20,75621,314 GALLATIN PIKE,MADISON,...,PT LOT 9 DOUGLAS-LEVINE,CIBORO CONSTRUCTION LLC,CASN,CAZ02A002,4041968,INSTALL A TYPE II ILLUMINATED STATIC BILLBOAR...,9,37010701,DONE,"{'latitude': '36.26534202', 'longitude': '-86...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,33306,2023048340,BUILDING COMMERCIAL - ROOFING / SIDING,"MULTIFAMILY, TRI-PLEX, QUAD, APARTMENTS",08215022700,2023-07-24,2023-08-12,16773,535 MAIN ST,NASHVILLE,...,LOT 2 VTH 5TH AND MAIN LLC PROPERTY & ACREAGE...,PATRICK WALSH,CACK,CAA03R398,4352470,TO CONDUCT ROOF REPAIRS TO EXISTING BUILDING. ...,5,37011900,ISSUED,"{'latitude': '36.174268995', 'longitude': '-86..."
458,33338,2023042603,BUILDING SIGN PERMIT,SIGN - GROUND / WALL SIGNS,05104006500,2023-06-28,2023-06-29,9400,900 GALLATIN PIKE S,MADISON,...,W SIDE GALLATIN PIKE N OF WEBSTER ST,JARVIS AWARD SIGN AND FLAG CO,CASN,CAZ02A001,4342864,TO ERECT/REPLACE 1 ILLUMINATED WALL SIGN & TO ...,9,37010701,ISSUED,"{'latitude': '36.253822016', 'longitude': '-86..."
459,33407,2023078671,BUILDING COMMERCIAL - REHAB,"AUTOMOBILE SALES, USED",08301003600,2023-10-13,2023-11-28,1,1200 GALLATIN AVE,NASHVILLE,...,LOT 1 RE-SUB PT LOTS 1 & 2 BLK A G. W. GIFFORD'S,SELF CONTRACTOR COMMERCIAL (SEE APPLICANT INFO...,CACR,CAF26S001,4398153,"TO CONTINUE USE AS AUTOMOBILE SALES, USED. USE...",5,37011700,ISSUED,"{'latitude': '36.190733987', 'longitude': '-86..."
460,33482,2023043270,BUILDING COMMERCIAL - REHAB,"GENERAL OFFICE, PROFESSIONAL SERVICES",03406000200,2023-06-30,2023-08-21,50000,1689 GALLATIN PIKE N,MADISON,...,LOT 2 RIVERGATE STATION SEC 1 2ND RESUB LOT 2,COMMERCIAL INSTALLATION AND CONSTRUCTION CO,CACR,CAD02B016,4343898,TO CONDUCT INTERIOR RENOVATIONS FOR TENANT; H&...,10,37010401,DONE,"{'latitude': '36.293865007', 'longitude': '-86..."


Need the coordinate data from the `mapped location` column

In [32]:
bdlg_permits['lat'] = [bdlg_permits['mapped_location'][x].get('latitude') for x in range(0, len(bdlg_permits))]
bdlg_permits['long'] = [bdlg_permits['mapped_location'][x].get('longitude') for x in range(0, len(bdlg_permits))]

In [33]:
bdlg_permits.to_csv('../data/clean/bldg_permits.csv')

...and Short Term Rental Permits<br><br>
Source: https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm/about_data

In [34]:
str_export = client.get("2z82-v8pm", limit=100000)
strs_raw = pd.DataFrame.from_records(str_export)
strs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15183 entries, 0 to 15182
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   permit                       15183 non-null  object
 1   applicant                    15168 non-null  object
 2   contact                      14787 non-null  object
 3   permit_subtype_description   15183 non-null  object
 4   permit_status                15183 non-null  object
 5   parcel                       15183 non-null  object
 6   date_entered                 15183 non-null  object
 7   date_issued                  13894 non-null  object
 8   expiration_date              15179 non-null  object
 9   address                      15183 non-null  object
 10  city                         15182 non-null  object
 11  state                        15182 non-null  object
 12  zip                          15183 non-null  object
 13  subdivision_lot              15

In [35]:
strs_raw = strs_raw.drop(strs_raw.columns[[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]], axis=1)

In [36]:
strs_raw['date_entered'] = pd.to_datetime(strs_raw['date_entered'])
strs_raw['date_issued'] = pd.to_datetime(strs_raw['date_issued'])
strs_raw['expiration_date'] = pd.to_datetime(strs_raw['expiration_date'])

In [37]:
strs_raw['applicant'] = strs_raw['applicant'].astype(str).str.upper()
strs_raw['contact'] = strs_raw['contact'].astype(str).str.upper()
strs_raw['permit_subtype_description'] = strs_raw['permit_subtype_description'].astype(str).str.upper()
strs_raw['permit_status'] = strs_raw['permit_status'].astype(str).str.upper()
strs_raw['address'] = strs_raw['address'].astype(str).str.upper()
strs_raw['city'] = strs_raw['city'].astype(str).str.upper()
strs_raw['state'] = strs_raw['state'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['purpose'] = strs_raw['purpose'].astype(str).str.upper()
strs_raw['permit_owner_name'] = strs_raw['permit_owner_name'].astype(str).str.upper()
strs_raw['permit_owner_addr2'] = strs_raw['permit_owner_addr2'].astype(str).str.upper()
strs_raw['permit_owner_city'] = strs_raw['permit_owner_city'].astype(str).str.upper()
strs_raw['permit_owner_state'] = strs_raw['permit_owner_state'].astype(str).str.upper()
strs_raw['permit_type'] = strs_raw['permit_type'].astype(str).str.upper()
strs_raw['permit_subtype'] = strs_raw['permit_subtype'].astype(str).str.upper()
strs_raw['council_dist'] = strs_raw['council_dist'].astype(str).str.upper()
strs_raw['census_tract'] = strs_raw['census_tract'].astype(str).str.upper()
strs_raw['permit_owner_addr1'] = strs_raw['permit_owner_addr1'].astype(str).str.upper()
strs_raw['permit_owner_addr3'] = strs_raw['permit_owner_addr3'].astype(str).str.upper()

In [38]:
rentals = strs_raw[strs_raw['zip'].isin(['37206', '37216', '37115'])].reset_index()
rentals

,index,permit,applicant,contact,permit_subtype_description,permit_status,parcel,date_entered,date_issued,expiration_date,...,permit_owner_city,permit_owner_state,permit_owner_zip,permit_type,permit_subtype,council_dist,census_tract,mapped_location,permit_owner_addr1,permit_owner_addr3
0,1,T2017008702,"TREHY, KEITH","TREHY, KEITH",SHORT TERM RENTAL - NON-OWNER OCCUPIED,EXPIRED,08304007300,2017-02-06,NaT,2017-05-08,...,NASHVILLE,TN,37206,CASR,CAZ10A002,7,37011600,"{'latitude': '36.189219', 'longitude': '-86.71...",NAN,NAN
1,2,2017055846,"HALE, JEFFREY L.",JEFFREY HALE,SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,08212026000,2017-08-25,2017-09-01,2024-09-01,...,HENDERSONVILLE,TN,37075,CASR,CAZ10A001,5,37011900,"{'latitude': '36.178000996', 'longitude': '-86...",NAN,NAN
2,16,2023047359,"OWNBEY, JASON","OWNBEY, JASON",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,09402023100,2023-07-19,2023-07-19,2024-07-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.166715015', 'longitude': '-86...",NAN,NAN
3,17,2022030944,"BURKART, JAMES","BURKART, JAMES",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,06114015300,2022-05-10,2022-05-10,2024-05-10,...,NASHVILLE,TN,37216,CASR,CAZ10A001,7,37011200,"{'latitude': '36.214595002', 'longitude': '-86...",NAN,NAN
4,21,2019021499,SARA CLINE,SARA CLINE,SHORT TERM RENTAL - OWNER OCCUPIED,EXPIRED,08208037200,2019-04-12,2019-04-19,2020-04-19,...,NASHVILLE,TN,37206,CASR,CAZ10A001,5,37011700,"{'latitude': '36.185369988', 'longitude': '-86...",NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2867,15128,2021051008,"11TH AVENUE PARTNERS, LLC",DWELL NASHVILLE,SHORT TERM RENTAL � MULTIFAMILY,ISSUED,082121D00200CO,2021-08-10,2021-08-13,2024-08-13,...,NASHVILLE,TN,37220,CASR,CAZ10A003,5,37011900,"{'latitude': '36.179004016', 'longitude': '-86...",BRAD THOMASON,NAN
2868,15130,2020002749,"STRATON, ANDREW & MARIKO","STRATON, ANDREW & MARIKO",SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08305036900,2020-01-14,2020-02-26,2023-02-26,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012100,"{'latitude': '36.184620017', 'longitude': '-86...",NAN,NAN
2869,15133,T2017030096,"NELSON, CANDICE & MATTHEW",CANDICE NELSON,SHORT TERM RENTAL - OWNER OCCUPIED,CANCELLED,08314008300,2017-05-05,NaT,2017-08-03,...,NASHVILLE,TN,37206,CASR,CAZ10A001,6,37012200,"{'latitude': '36.174626987', 'longitude': '-86...",NAN,NAN
2870,15144,2023037519,"MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL","MOORE, JONATHAN E. & DONNA KIMBERLY & ET AL",SHORT TERM RENTAL - OWNER OCCUPIED,ISSUED,05207008600,2023-06-07,2023-06-07,2024-06-07,...,MADISON,TN,37115,CASR,CAZ10A001,9,37010601,"{'latitude': '36.247076009', 'longitude': '-86...",NAN,NAN


Need the coordinate data from the `mapped location` column

In [39]:
rentals['lat'] = [rentals['mapped_location'][x].get('latitude') for x in range(0, len(rentals))]
rentals['long'] = [rentals['mapped_location'][x].get('longitude') for x in range(0, len(rentals))]

In [40]:
rentals.to_csv('../data/clean/rentals.csv')

...and the pedestrian signal inventory<br><br>
Source: https://data.nashville.gov/Transportation/Pedestrian-Signal-Inventory-and-ADA-Self-Assessmen/6xet-f7u7/about_data

In [41]:
ped_sig = client.get("6xet-f7u7", limit=1000000)
ped_inv = pd.DataFrame.from_records(ped_sig)
ped_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3153 entries, 0 to 3152
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   intersectionid               3153 non-null   object
 1   onst                         3153 non-null   object
 2   crst                         3153 non-null   object
 3   quad                         3151 non-null   object
 4   inspector                    3153 non-null   object
 5   date                         3153 non-null   object
 6   evnt_lat                     3136 non-null   object
 7   evnt_lon                     3136 non-null   object
 8   evnt_map                     3153 non-null   object
 9   evnt_type                    3150 non-null   object
 10  ped_signal_pres              3150 non-null   object
 11  oa_cmnt                      3150 non-null   object
 12  mapped_location              3136 non-null   object
 13  :@computed_region_wvby_4s8j  3136

In [42]:
ped_inv = ped_inv.drop(ped_inv.columns[[13, 14, 15, 16]], axis=1)

In [43]:
ped_inv['onst'] = ped_inv['onst'].astype(str).str.upper()
ped_inv['crst'] = ped_inv['crst'].astype(str).str.upper()
ped_inv['quad'] = ped_inv['quad'].astype(str).str.upper()
ped_inv['inspector'] = ped_inv['inspector'].astype(str).str.upper()
ped_inv['evnt_type'] = ped_inv['evnt_type'].astype(str).str.upper()

In [44]:
ped_inv = ped_inv[(ped_inv['onst'].str.contains('GALLATIN|MAIN')) | (ped_inv['crst'].str.contains('GALLATIN|MAIN'))]

In [45]:
ped_inv['date'] = pd.to_datetime(ped_inv['date'])

In [46]:
ped_inv.to_csv('../data/clean/ped_inv.csv')